In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('individual_stocks_5yr/individual_stocks_5yr/MSFT_data.csv')

In [4]:
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,27.35,27.71,27.31,27.55,33318306,MSFT
1,2013-02-11,27.65,27.92,27.50,27.86,32247549,MSFT
2,2013-02-12,27.88,28.00,27.75,27.88,35990829,MSFT
3,2013-02-13,27.93,28.11,27.88,28.03,41715530,MSFT
4,2013-02-14,27.92,28.06,27.87,28.04,32663174,MSFT


In [5]:
df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d')
df.set_index('date', inplace = True)

In [6]:
df.tail()

,open,high,low,close,volume,Name
date,,,,,,
2018-02-01,94.79,96.070,93.5813,94.26,47227882,MSFT
2018-02-02,93.64,93.970,91.5000,91.78,47867753,MSFT
2018-02-05,90.56,93.240,88.0000,88.00,51031465,MSFT
2018-02-06,86.89,91.475,85.2500,91.33,67998564,MSFT
2018-02-07,90.49,91.770,89.2000,89.61,41107592,MSFT


In [7]:
test_split = '2017-01-01'
train = df[:test_split]
test = df[test_split:]

In [8]:
len(test)/(len(train)+len(test))*100

22.00158856235107

In [4]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import random

class Agent:
    
    def __init__(self):

        self.actions = 3           
        self.curr_inventory = []
        self.gamma = 0.95
        self.trade_log = deque(maxlen = 5000)
        self.model = model()

    def _model(self):
        # build model here
        model = Sequential()
        model.add(Dense(activation = "relu"))
        model.add(Dense(units=self.actions))
        model.compile(loss = 'huber_loss', optimizer = 'adam')
        return model

    def history(self, attributes):
        self.trade_log.append(attributes)

    def take_action(self, state):
        proba = self.model.predict(state)
        return np.argmax(proba[0])

    def train(self, batch_size):
        mini_batch = random.sample(self.trade_log, batch_size)
        
        x_train , y_train = []
        
        for state, action, reward, next_state in mini_batch:
            target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
        q_values = self.model.predict(state)
        q_values[0][action] = target
        
        x_train.append(state[0])
        y_train.append(q_values[0])
        
        loss = self.model.fit(np.array(x_train), np.array(y_train), epochs = 1).history['loss'][0]
        
        return loss